Msc Data Science and Economics

Università degli Studi di Milano

Martina Viggiano (954603)

## *Perspectives on data sharing: a social media analysis*

# **1. Data Import and Cleaning Notebook**

Summary:

1.   INITIALIZE
2.   DATA EXTRACTION
    * 2.1 Define topics, write queries
    * 2.2 Serialize data
    * 2.3 Amount of tweets collected
3. DATA CLEANING
    * 3.1 Load back data
    * 3.2 Clean data based on words, author ID, and number of tags
    * 3.3 Extract hashtags
    * 3.4 Extract tags
    * 3.5 Extract websites
    * 3.6 Cleaning code
    * 3.7 Number of conversations
4. PRE AND POST COVID DATA
5.  FILTERING DATA
    * 5.1 Filter by most frequent users
    * 5.2 Filter by keywords: research and academy terms
    * 5.3 Combine dfs of the 2 previous results and drop duplicates
6. TIME SERIES
7. FIRST TWEETS
8.PRE-PROCESSING DATA FILTERED BY KEYWORDS




---

## 1. **INITIALIZE**


We connect the notebook to our personal Google Drive folder: it will be used to store and load data.

In [ ]:
# Connect Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Università/Thesis/')

We import and download libraries and packages.

In [ ]:
# install tweepy
!pip install tweepy==4.4.0

In [ ]:
# install unidecode
!pip install -U spacy unidecode

In [ ]:
# install spacy and download english language
!python -m spacy download en_core_web_sm

In [ ]:
!pip install langdetect

In [ ]:
# Import libraries
import pickle
import tweepy
from tweepy.auth import OAuthHandler
import numpy as np
import pandas as pd
import datetime
import re
from unidecode import unidecode
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import statistics
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import datetime
from tqdm.notebook import tqdm
tqdm.pandas()
import langdetect
from langdetect import detect

## 2. **DATA EXTRACTION**


### 2.1 Define topics, write queries

We are looking for tweets which text contains:

*   #Opendata
*   opendata
*   open data
*   #Datasharing
*   datasharing
*   data sharing
*   #Openscience
*   openscience
*   open science
*   #Openresearch
*   open research

Moreover, for each query (topic) we extract a separate df in which we kept only tweets published by verified users.

To reduce chances to extract false positives (tweets which are not relevant to our purpose), we further filter with respect to some conditions, based on terms and phares found in the collected tweets.

In particular, we excluded twees containing:
* media
* videos
* images
* links
* geolocation
* a list of terms, lemmas and phrases.

These conditions have been defined after multiple scans on the tweets retrieved by the queries.

In [ ]:
client = tweepy.Client(bearer_token="key")

In [ ]:
output = []
for tweet in tweepy.Paginator(client.search_all_tweets,
                              query=' #opendata -is:retweet -is:nullcast lang:en -has:media -has:videos -has:images -has:geo -has:links -gov -freelanc -marvel -rush -faq -hire -javascript -#EUOpenDataDays -pollution -plastic -"check out" -(hi there) -(dm is open) -"read more" -"this article" -"is out!" -"police have shot" -"open access ucl" -"freelance" -"webinar" -"open for submission" -"happy new" -"open for commission" -"infected" -"anti-science" -"still open" -"bbc" -"episode" -"podcast" -"hiring" -"dm open" -"open the school" -"birthday" -"follow" -"fauci" -"biden" -"@CopernicusEU" -"cool" -"congrat" -"map" -"viral" -"lol" -"lmao" -"MusicHackFest" -"need help" -"#smartcities" -"work in progress" -"sex" -"gov"',
                              start_time="2021-01-01T00:00:00Z",
                              end_time="2021-12-31T00:00:00Z",
                              tweet_fields=['created_at', 'conversation_id'],
                              expansions = ['author_id', 'in_reply_to_user_id'],
                              max_results=400).flatten(limit=2000):

    output.append({'text': tweet.text, 'author_id': tweet.author_id, 'tweet_id': tweet.id, 'date': tweet.created_at, 'conversation_id': tweet.conversation_id, 'reply_to_user_id': str(tweet.in_reply_to_user_id) } )

In [ ]:
df_h_opendata_21 = pd.DataFrame(output)
df_h_opendata_21

In [ ]:
data_merge_21 = pd.concat(
                        [df_h_opendata_21, df_h_openresearch_21, df_h_openscience_21, df_h_datasharing_21,
                         df_opendata_21, df_openresearch_21, df_openscience_21, df_datasharing_21,
                         df_open_data_21, df_open_research_21, df_open_science_21, df_data_sharing_21,
                         df_reasearch_data_share_21]
                       ).sort_values('date').drop_duplicates().reset_index(drop=True).drop_duplicates(subset=['text'], keep='first')

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/data_merge_21.pkl", "wb") as f:
    pickle.dump(data_merge_21, f)

We populate a dataframe with: the text of the tweet, the author ID, the tweet ID, the publication date, the conversation ID, the ID of the parent Tweet's author.

In [ ]:
list_df_by_year = [data_merge_21, data_merge_20, data_merge_19, data_merge_18, data_merge_17, data_merge_16, data_merge_15, data_merge_14, data_merge_13, data_merge_12, data_merge_11, data_merge_10, data_merge_09, data_merge_08, data_merge_07]
list_years = []
list_counts = []
for df in list_df_by_year:
    list_years.append(df['date'][0].year)
    list_counts.append(df.shape[0])

In [ ]:
df_count_per_year = pd.DataFrame(list(zip(list_years, list_counts)), columns =['year', 'count'])

In [ ]:
df_count_per_year = df_count_per_year.iloc[::-1].reset_index(drop=True)

### 2.2 Serialize data

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/df_count_per_year.pkl", "wb") as f:
    pickle.dump(df_count_per_year, f)

### 2.3 Amount of tweets collected

We merge the dfs by year - from 2007 to 2021 - dropping duplicates (entire rows) and dropping rows with identical texts.

By merging all the dfs togetherwe obtained:

In [ ]:
data_merge_all_years = pd.concat(
                        [data_merge_07, data_merge_08, data_merge_09, data_merge_10, data_merge_11, data_merge_12, data_merge_13, data_merge_14, data_merge_15, data_merge_16, data_merge_17, data_merge_18, data_merge_19, data_merge_20, data_merge_21]
                       ).sort_values('date').drop_duplicates().reset_index(drop=True).drop_duplicates(subset=['text'], keep='first')
data_merge_all_years.shape[0] 

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/data_merge_all_years.pkl", "wb") as f:
    pickle.dump(data_merge_all_years, f)

---

## 3. **DATA CLEANING**


### 3.1 Load back data

We first re-load the "full-data" dataframes.

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/data_merge_all_years.pkl", "rb") as f:
    data_merge_all_years = pickle.load(f)

In [ ]:
data_merge_all_years.shape[0]

485268

In [ ]:
data_merge = data_merge_all_years

### 3.2 Clean data based on words, author ID, and number of tags

We clean the resulting dfs, by excluding tweets containing some phrases and words.

By cleaning we reduce the total number of rows from 485268 to 224784
.

In [ ]:
remove_tweets = ['freelance', 'webinar', 'open for submission', 'happy new', 'open for commission',  'anti-science', 'still open', 'bbc', 'episode', 'podcast', 'hiring', 'dm open', 'PLOS', 'AISRT', 'fauci', 'open the school', 'birthday', 'thank', '@CopernicusEU', 'cool', 'congrat', 'map', '@EU_opendata', 'link', ' gov', 'music', 'trump', 'good morning', 'smartcit', 'smart cit', 'minister', ' rap ', 'boutique', 'opengov', 'police', ' kid ', ' kids ', 'uganda', 'blockchain', 'twitter', 'welcome ', 'webinar', 'workshop', ' site ', 'opengov', 'polit', 'google', 'website', 'jackson', ' city ', ' pic ', 'picture', 'tomorrow', 'nope', ' tax',  'country', 'global', 'CyberAlertHigh', 'pyhton', 'python', 'fuck', ' photo', 'happy 2017', 'population',  ' rich ', 'amazon ', 'apple ', 'mobile', 'wifi', 'corruption', 'somali', 'we need your help', 'stay tuned',  'help', 'purchas', 'geek', 'quote', 'christmas', 'today', 'firefox', 'foxnews', 'tanzania', 'browser', ' beta ', 'all the best', 'meeting up',  "i'm looking forward", 'presentation', 'come work', 'ghana', 'citizen', ' dear ',  'geogr', '#localgov', 'bureau', 'cities', 'bugs', 'judge', 'govlive', 'nsa ', 'govt', 'Copernicus', 'excited to be', ' ios ', 'postcode', 'thinking about', 'registe', 'bigot', 'cities', 'bank', 'pdf', 'developer', 'dept',  'german', 'pleased to be', 'region', 'pm', 'agenda', 'white', 'follower', 'tracker', 'news', 'transparency', 'speaker', ' sign ', ' signed ', 'signup', ' elect', 'ottawa', '#iot', 'hack', 'chatting', 'look forward', 'looking forward', 'fwrd', 'real time', 'opendatato', '#gov', '#iog', 'patient', 'workshop', 'marc', 'facebook', 'squirt', 'slurp', 'caffein', 'reuters', 'app', 'transparent', 'mountain', ' fan ', 'ventura', 'new cases', ' crypto', 'california', '#okfest', '#opengov', 'africa', 'salary', 'testkit', 'mylab', 'icmr', 'india',  'findora', 'tutorial', ' opt ', 'poem', 'government', 'odifridays', 'java', ' gun', 'LD 1910', 'eureka']#summit, public health, 'infected', 'EO_OPEN_SCIENCE', 'peer','cloud', 'regulation','summit', 'policy', 'conference', # [' LA ', 'NYC'], ' uk ', 'seminar', ' new york ', ' newyork', 'toronto',

In [ ]:
data_merge_clean = data_merge.loc[~data_merge['text'].str.contains('|'.join(remove_tweets), case=False)]

We drop tweets by author id. The list has been determined after a manual scan.

In [ ]:
remove_author_id = [3378865745, 2895978157, 2925060501, 132737300, 352650591, 304572237, 352650591, 14739956, 118012929, 300036810, 1115904375518576640, 967517677249064960, 976538112, 2793987281, 2599797763]

In [ ]:
data_merge_clean = data_merge_clean.loc[~data_merge_clean['author_id'].isin(remove_author_id)]

We drop tweets in which the number of tags is higher than the number of words.

In [ ]:
def less_tags(text):
    lista_ok= []
    lista_tag= []
    num_tags = [l for l in text.split(' ') if len(l) > 0 and l[0]=='@']
    num_not = [l for l in text.split(' ') if len(l) > 0 and l[0]!='@' and l[0]!='#']
    if len(num_tags) < len(num_not):
        return text
    else:
        return 0

In [ ]:
less_tags = data_merge_clean["text"].progress_apply(lambda text: less_tags(text))

  0%|          | 0/233655 [00:00<?, ?it/s]

In [ ]:
drop_list_tag = less_tags[less_tags==0].index

In [ ]:
data_merge_clean = data_merge_clean[~data_merge_clean.index.isin(drop_list_tag)]

In [ ]:
data_merge_preclean = data_merge_clean

In [ ]:
data_merge_preclean.shape[0]

231226

Save cleaned df merge excluding rows extracted in previous data collection.

In [ ]:
# Save merged df - full data
with open("/content/drive/MyDrive/Università/Thesis/data_merge_preclean.pkl", "wb") as f:
    pickle.dump(data_merge_preclean, f)

If needed, load back dfs.

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/data_merge_preclean.pkl", "rb") as f:
    data_merge_preclean = pickle.load(f)

In [ ]:
data_merge = data_merge_preclean

### 3.3 Extract hashtags

We define a function to extract hashtags (#) found in each tweets and append them in a new column named "hashtags".

In [ ]:
def extract_hashtags(text):
     
    # initializing hashtag_list variable
    hashtag_list = []

    for tweet in text: 
        # splitting the text into words
        for word in text.split():
            
            # checking the first character of every word
            if word[0] == '#':
                
                # adding the word to the hashtag_list
                if word[1:].lower() not in hashtag_list:
                    hashtag_list.append(word[1:].lower())
    
    hashtag_str = ' '.join(hashtag_list)

    return hashtag_str

In [ ]:
data_merge['hashtags'] = [extract_hashtags(x) for x in data_merge.text]

In [ ]:
data_merge['hashtags'] = [x.replace(',','').replace('.','').replace('!','').replace('?','').replace(':','') for x in data_merge['hashtags'] ]

In [ ]:
# Number of tweets without hashtags
len(data_merge[data_merge['hashtags'] == '']) # 95745

101546

In [ ]:
# Number of unique hastags
unique_hashtags = []
for row in data_merge['hashtags']:
    for x in row.split(' '):
        if x not in unique_hashtags:
            unique_hashtags.append(x)
len(unique_hashtags) # 42552

45385

### 3.4 Extract tags

We define a function to extract tags (@) found in each tweets and append them in a new column named "tags".

In [ ]:
def extract_tags(text):
     
    # initializing tag list variable
    tag_list = []

    for tweet in text: 
        # splitting the text into words
        for word in text.split():
            
            # checking the first character of every word
            if word[0] == '@':
                
                # adding the word to the hashtag_list
                if word[1:].lower() not in tag_list:
                    tag_list.append(word[1:].lower())

    return tag_list

In [ ]:
data_merge['tags'] = [' '.join(extract_tags(x)).strip() for x in data_merge.text]
data_merge['n_tags'] = [len(extract_tags(x)) for x in data_merge.text]

In [ ]:
# Number of tweets without tags - full data
len(data_merge[data_merge['tags'] == '']) # 78126

84306

### 3.5 Extract websites

We define a function to extract links (http and www) found in each tweets and append them in a new column named "websites".

In [ ]:
def extract_links(text):
     
    # initializing site_list variable
    site_list = []

    for tweet in text: 
        for word in text.split():
        
            if word.startswith("http")== True or word.startswith("www")== True:

            # adding only new word to the website list
                if word.lower() not in site_list:
                    site_list.append(word.lower())
        
    site_str = ' '.join(site_list)

    return site_str

In [ ]:
data_merge['websites'] = [extract_links(x) for x in data_merge.text]

In [ ]:
# Number of tweets without website links - full data
len(data_merge[data_merge['websites'] == '']) # 215028

230712

### 3.6 Cleaning code

We define a new column containing cleaned tweets: we cleaned textual data by removing contractions and taking off links, tags, punctuation and stopwords.

Furthermore, we detect language of each tweet by using *langdetect* library: in this way, we are able to drop out tweets that are not written in english.

At the end of cleaning, is there is no remaining lemma in the new column *text_clean*, we drop the row.

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def full_text_clean(text):

    text = text.lower()

    elem = (
        text.replace("don't",'do not')
        .replace("can't",'cannot')
        .replace("&amp;", "and")
        .replace("'re", " are")
        .replace(" open data ", " opendata ")
        .replace(" data sharing ", " datasharing ")
        .replace(" data share ", " datasharing ")
        .replace(" datashare ", " datasharing ")
        .replace(" open science ", " openscience ")
        .replace(" open research ", " openresearch ")
        .replace("#oa", "#openaccess")
        .replace(" oa ", " openaccess ")
        .replace("meta data", "metadata")
        .replace(" iso ", "")
        .replace("etc", "")
        )

    elem = unidecode(re.sub(' +', ' ', elem.strip())) #any additional whitespaces and foreign characters
    elem = elem.strip()
    elem = elem.lower()
    elem = re.sub("@[A-Za-z0-9_]+","", elem) # no tags
    elem = re.sub("#([a-zA-Z0-9_]{1,50})", "", elem) # no hashtags
    elem = re.sub(r"http\S+", "", elem) # no links
    elem = re.sub(r"www.\S+", "", elem) # no links
    elem = re.sub('[()!?]', ' ', elem) # no punctuation
    elem = re.sub('\[.*?\]',' ', elem) # no punctuation
    elem = re.sub("[^a-z0-9]"," ", elem) # no alphanumeric 

    elem = unidecode(re.sub(' +', ' ', elem.strip()))
    elem = elem.strip()

    if elem.strip() == '' or pd.isna(elem):
        return ''

    result = []

    if detect(elem) == 'en':   

        doc = nlp(elem)

        
        for token in doc:
            if (len(token.text) > 1 
                and token.text.isalpha() # Token is word
                and token.pos_ not in ['NUM']   # Token not NUM, PROPN nor ADV,, , 'ADV', 'PRON', 'CONJ', 'PROPN']
                and not token.is_punct # Token not punctuation
                and not token.is_stop # Token not stopword
            ):

                result.append(token.lemma_.lower().replace("datum", "data").replace("medium", "media").replace("datashare", "datasharing"))
    
    result = " ".join([i for i in result if len(i) > 2 ])
    
    return result

In [ ]:
data_merge['text_clean'] = data_merge["text"].progress_apply(lambda x: full_text_clean(x)) # 1h 12min

  0%|          | 0/231226 [00:00<?, ?it/s]

After cleaning, the resulting dataframe contains 209160 rows.

In [ ]:
data_merge_text_clean_nlp = data_merge.drop(data_merge[data_merge.text_clean ==''].index)

Then, we create a new column with text cleaned excluding terms we explicitly defined as constraint in the extraction phase, namely: *data, research, sharing, open, science, opendata, openresearch, datasharing, openscience*.

In [ ]:
def remove_top_terms(text):
    
    top_terms = ['data', 'research', 'sharing', 'open', 'science', 'opendata', 'openresearch', 'datasharing', 'openscience']

    querywords = text.split()

    resultwords  = [word for word in querywords if word.lower() not in top_terms]
    
    result = " ".join([i for i in resultwords if len(i) > 2 ])
    
    return result

In [ ]:
data_merge_text_clean_nlp['text_clean_notop'] = data_merge_text_clean_nlp["text_clean"].progress_apply(lambda x: remove_top_terms(x))

  0%|          | 0/224784 [00:00<?, ?it/s]

Then, we extract the year of publication of each tweet.

In [ ]:
data_merge_text_clean_nlp['year'] = data_merge_text_clean_nlp['date'].dt.year

In [ ]:
# Save text clean NLP df - full data 
with open("/content/drive/MyDrive/Università/Thesis/data_merge_text_clean_nlp.pkl", "wb") as f:
    pickle.dump(data_merge_text_clean_nlp, f)

In [ ]:
data_merge_text_clean_nlp.shape[0]

224784

Load back, if needed.

In [ ]:
# Load back text clean NLP df - full data 
with open("/content/drive/MyDrive/Università/Thesis/data_merge_text_clean_nlp.pkl", "rb") as f:
    data_merge_text_clean_nlp = pickle.load(f)

In [ ]:
data_merge = data_merge_text_clean_nlp 

### 3.7 Number of conversations

We have 204771 different conversations inside our 224784-row df.

In [ ]:
len(data_merge['conversation_id'].unique())

204771

In [ ]:
conversations_df = pd.DataFrame(data_merge['conversation_id'].value_counts())

In [ ]:
conversations_list = data_merge['conversation_id'].value_counts().loc[lambda x : x>1].index.tolist()

In [ ]:
data_merge_conversations = data_merge[data_merge['conversation_id'].isin(conversations_list)]

In [ ]:
# Save conversations df - full data
with open("/content/drive/MyDrive/Università/Thesis/data_merge_conversations.pkl", "wb") as f:
    pickle.dump(data_merge_conversations, f)

---

## 4. **PRE AND POST COVID DATA**
We split data betweet pre and post Covid19 era.

In particular, we consider Pre-Covid19 era tweets published in 2018 and 2019, while we consider Post-Covid19 era tweets published in 2020 and 2021.

In the pre-Covid19 era we collected 36706 tweets and in the post-Covid19 era we collected 35574 tweets.

In [ ]:
data_pre_covid = data_merge[(data_merge['date'].dt.year==2018)|(data_merge['date'].dt.year==2019)]
data_post_covid = data_merge[(data_merge['date'].dt.year==2020)|(data_merge['date'].dt.year==2021)]

In [ ]:
data_pre_covid.shape[0], data_post_covid.shape[0]

(36706, 35574)

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/data_pre_covid.pkl", "wb") as f:
    pickle.dump(data_pre_covid, f)
with open("/content/drive/MyDrive/Università/Thesis/data_post_covid.pkl", "wb") as f:
    pickle.dump(data_post_covid, f)

---

## 5. **FILTERING DATA**

By joining the data extracted from the queries, we obtain a df with 224784 rows.

In [ ]:
data_merge.shape[0]  # 224784

224784

Thus, we have to find a way to further filter the df in order to reduce the number of FP.

We define several ways for filtering the tweets collected, in order to extract relevant tweets.

1.   Filter by most frequent users (17545 rows: 8%)
2.   Filter by keywords (87256 rows: 39%)
3.   Mix all the result above and drop duplicates (97645 rows: 43%)



### 5.1 Filter by most frequent users

We take the tweets written by top 100 users most present in data collected.

In [ ]:
len(data_merge['author_id'].unique()) # we have 90934 different users

90934

In [ ]:
frequent_users = data_merge['author_id'].value_counts()[:100].sort_values(ascending=False).index.tolist() 

In [ ]:
filtered_user_df = data_merge[data_merge['author_id'].isin(frequent_users)]

In [ ]:
filtered_user_df.shape[0] # 17545 rows: 8% of total rows

17545

In [ ]:
# Save filered by users df
with open("/content/drive/MyDrive/Università/Thesis/filtered_user_df.pkl", "wb") as f:
    pickle.dump(filtered_user_df, f)

### 5.2 Filter by keywords: research and academy terms

We selected following relevant keywords "research", "university", "academy", "publish", "professor", "availability", "PhD", "discovery", "reproducibility", "scientific community", "scient". We filtered for tweets containing their lemmas.

In [ ]:
# Function for searching tweets with given term / hashtag or retrieving the amount of tweets meeting the condition
def search(df, column, term, number=False):
    filtered = df[df[column].str.contains(term, case=False)] 
    if number==False:
        return filtered
    else:
        return len(filtered)

In [ ]:
filtered_k_university_df = search(data_merge, 'text', 'universit') # 1463 rows
filtered_k_university_df.shape[0]

1463

In [ ]:
filtered_k_academ_df = search(data_merge, 'text', 'academ') # 2827 rows
filtered_k_academ_df.shape[0]

2827

In [ ]:
filtered_k_research_df = search(data_merge, 'text', 'research') # 21750  rows
filtered_k_research_df.shape[0]

21750

In [ ]:
filtered_k_publish_df = search(data_merge, 'text', 'publi') # 18269 rows
filtered_k_publish_df.shape[0]

18269

In [ ]:
filtered_k_prof_df = search(data_merge, 'text', 'prof') # 3200 rows
filtered_k_prof_df.shape[0]

3200

In [ ]:
filtered_k_available_df = search(data_merge, 'text', 'availab') # 4802 rows
filtered_k_available_df.shape[0]

4802

In [ ]:
filtered_k_phd1_df = search(data_merge, 'text', 'phd') # 1493 rows
filtered_k_phd1_df.shape[0]

1493

In [ ]:
filtered_k_phd2_df = search(data_merge, 'text', 'p.h.d') # 22 rows
filtered_k_phd2_df.shape[0]

22

In [ ]:
filtered_k_discovery_df = search(data_merge, 'text', 'discover') # 1229 rows
filtered_k_discovery_df.shape[0]

1229

In [ ]:
filtered_k_scientificommunity_df = search(data_merge, 'text', 'scientific community') # 139 rows
filtered_k_scientificommunity_df.shape[0]

139

In [ ]:
filtered_k_reproducibility_df = search(data_merge, 'text', 'reproducib') # 1883 rows
filtered_k_reproducibility_df.shape[0]

1883

In [ ]:
filtered_k_scien_df = search(data_merge, 'text', 'scien') # 56977 rows
filtered_k_scien_df.shape[0]

56977

In [ ]:
filtered_keywords_df = pd.concat([filtered_k_university_df, filtered_k_academ_df , filtered_k_research_df, filtered_k_publish_df , filtered_k_prof_df , filtered_k_available_df, filtered_k_phd1_df, filtered_k_phd2_df, filtered_k_discovery_df, filtered_k_scientificommunity_df, filtered_k_reproducibility_df, filtered_k_scien_df
                              ]).sort_values('date').drop_duplicates().reset_index(drop=True).drop_duplicates(subset=['text'], keep='first')

In [ ]:
filtered_keywords_df.shape[0] # 87256 rows: 39% of total rows

87256

In [ ]:
# Save filered by keywords df
with open("/content/drive/MyDrive/Università/Thesis/filtered_keywords_df.pkl", "wb") as f:
    pickle.dump(filtered_keywords_df, f)

### 5.3 Combine dfs of the 2 previous results and drop duplicates

We joined the filtered data-frames obtained by using the ways above.

In [ ]:
filtered_user_df.shape[0] + filtered_keywords_df.shape[0] # simply summing rows without dropping duplicates

104801

In [ ]:
filtered_data_mix_df = pd.concat([filtered_user_df, filtered_keywords_df
                              ]).sort_values('date').drop_duplicates().reset_index(drop=True).drop_duplicates(subset=['text'], keep='first')

In [ ]:
filtered_data_mix_df.shape[0] # 57928 rows: 28%

97645

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/filtered_data_mix_df.pkl", "wb") as f:
    pickle.dump(filtered_data_mix_df, f)

---

## 6. **TIME SERIES**
We collect tweet counts from 2007 to 2021 containing the following terms/hashtags: *opendata, openscience, openresearch, datasharing*.

We build one table for absolute frequencies and one for relative frequencies.

Please note, we did not put any constraint on language: we did not filter for english language only.

In [ ]:
client = tweepy.Client(bearer_token="key")

In [ ]:
results = []

result = client.get_all_tweets_count(
    query='#openresearch',
    start_time=datetime.datetime(2020, 1, 1),
    end_time=datetime.datetime(2020, 12, 31),
    granularity="day")

results.append(result)

In [ ]:
next_token = ""

if "next_token" in result.meta:
    next_token = result.meta["next_token"]

In [ ]:
stop = False

i = 2 # Because we have already got the first page

while not stop:
    print("Downloading page: " + str(i))

    result = client.get_all_tweets_count(
        query='#openresearch',
        start_time=datetime.datetime(2020, 1, 1),
        end_time=datetime.datetime(2020, 12, 31),
        granularity="day",
        next_token=next_token
    )

    results.append(result)

    i += 1

    if "next_token" in result.meta:
        next_token = result.meta["next_token"]

        if next_token is not None and next_token != "":
            stop = False
        else:
            stop
    else:
        stop = True


In [ ]:
sum = 0
for x in results:
    sum +=  x.meta.get('total_tweet_count')
sum

In [ ]:
output = []
for result in results:
    for tweet in result.data:
        output.append({'start': tweet['start'], 'end': tweet['end']} ) 

In [ ]:
[d['start'] for d in output][0][:4]

In [ ]:
df_time_count = pd.DataFrame(np.array([[[d['start'] for d in output][0][:4], sum]]), columns=['year', 'count'])

In [ ]:
df_time_count = df_time_count.append(dict(zip(df_time_count.columns,[[d['start'] for d in output][0][:4], sum])), ignore_index=True)

In [ ]:
#df_time_count_openresearch = df_time_count
df_time_count_open_research = df_time_count_open_research.drop_duplicates(subset ="year").reset_index(drop=True)
df_time_count_open_research

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_open_research.pkl", "wb") as f:
    pickle.dump(df_time_count_open_research, f)

Load back dfs

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_h_opendata.pkl", "rb") as f:
    df_time_count_h_opendata = pickle.load(f)
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_opendata.pkl", "rb") as f:
    df_time_count_opendata = pickle.load(f)
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_open_data.pkl", "rb") as f:
    df_time_count_open_data = pickle.load(f)

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_h_openscience.pkl", "rb") as f:
    df_time_count_h_openscience = pickle.load(f)
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_openscience.pkl", "rb") as f:
    df_time_count_openscience = pickle.load(f)
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_open_science.pkl", "rb") as f:
    df_time_count_open_science = pickle.load(f)

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_h_openresearch.pkl", "rb") as f:
    df_time_count_h_openresearch = pickle.load(f)
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_openresearch.pkl", "rb") as f:
    df_time_count_openresearch = pickle.load(f)
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_open_research.pkl", "rb") as f:
    df_time_count_open_research = pickle.load(f)

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_h_datasharing.pkl", "rb") as f:
    df_time_count_h_datasharing = pickle.load(f)
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_datasharing.pkl", "rb") as f:
    df_time_count_datasharing = pickle.load(f)
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_data_sharing.pkl", "rb") as f:
    df_time_count_data_sharing = pickle.load(f)

We merge dfs by type.

In [ ]:
df_time_count_hashtags = pd.concat([df_time_count_h_opendata['year'], df_time_count_h_opendata['count'], df_time_count_h_openscience['count'], df_time_count_h_openresearch['count'], df_time_count_h_datasharing['count']], axis=1, 
                                   keys=['year', '#opendata', '#openscience', '#openresearch', '#datasharing']).astype(str).astype(int)                        

In [ ]:
df_time_count_oneterm = pd.concat([df_time_count_opendata['year'], df_time_count_opendata['count'], df_time_count_openscience['count'], df_time_count_openresearch['count'], df_time_count_datasharing['count']], axis=1, 
                                   keys=['year', 'opendata', 'openscience', 'openresearch', 'datasharing']).astype(str).astype(int)

In [ ]:
df_time_count_twoterms = pd.concat([df_time_count_open_data['year'], df_time_count_open_data['count'], df_time_count_open_science['count'], df_time_count_open_research['count'], df_time_count_data_sharing['count']], axis=1, 
                                   keys=['year', 'open data', 'open science', 'open research', 'data sharing']).astype(str).astype(int)

We serialize dfs

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_hashtags.pkl", "wb") as f:
    pickle.dump(df_time_count_hashtags, f)
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_oneterm.pkl", "wb") as f:
    pickle.dump(df_time_count_oneterm, f)
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_twoterms.pkl", "wb") as f:
    pickle.dump(df_time_count_twoterms, f)

We compute relative frequencies.

In [ ]:
df_time_count_hashtags_relative = df_time_count_hashtags.copy()
df_time_count_oneterm_relative = df_time_count_oneterm.copy()
df_time_count_twoterms_relative = df_time_count_twoterms.copy()

In [ ]:
df_time_count_hashtags_relative['#opendata'] = ((df_time_count_hashtags['#opendata'] /  df_time_count_hashtags['#opendata'].sum()) * 100).round(2)
df_time_count_hashtags_relative['#openscience'] = ((df_time_count_hashtags['#openscience'] /  df_time_count_hashtags['#openscience'].sum()) * 100).round(2)
df_time_count_hashtags_relative['#openresearch'] = ((df_time_count_hashtags['#openresearch'] /  df_time_count_hashtags['#openresearch'].sum()) * 100).round(2)
df_time_count_hashtags_relative['#datasharing'] = ((df_time_count_hashtags['#datasharing'] /  df_time_count_hashtags['#datasharing'].sum()) * 100).round(2)

In [ ]:
df_time_count_oneterm_relative['opendata'] = ((df_time_count_oneterm_relative['opendata'] /  df_time_count_oneterm_relative['opendata'].sum()) * 100).round(2)
df_time_count_oneterm_relative['openscience'] = ((df_time_count_oneterm_relative['openscience'] /  df_time_count_oneterm_relative['openscience'].sum()) * 100).round(2)
df_time_count_oneterm_relative['openresearch'] = ((df_time_count_oneterm_relative['openresearch'] /  df_time_count_oneterm_relative['openresearch'].sum()) * 100).round(2)
df_time_count_oneterm_relative['datasharing'] = ((df_time_count_oneterm_relative['datasharing'] /  df_time_count_oneterm_relative['datasharing'].sum()) * 100).round(2)

In [ ]:
df_time_count_twoterms_relative['open data'] = ((df_time_count_twoterms_relative['open data'] /  df_time_count_twoterms_relative['open data'].sum()) * 100).round(2)
df_time_count_twoterms_relative['open science'] = ((df_time_count_twoterms_relative['open science'] /  df_time_count_twoterms_relative['open science'].sum()) * 100).round(2)
df_time_count_twoterms_relative['open research'] = ((df_time_count_twoterms_relative['open research'] /  df_time_count_twoterms_relative['open research'].sum()) * 100).round(2)
df_time_count_twoterms_relative['data sharing'] = ((df_time_count_twoterms_relative['data sharing'] /  df_time_count_twoterms_relative['data sharing'].sum()) * 100).round(2)

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_hashtags_relative.pkl", "wb") as f:
    pickle.dump(df_time_count_hashtags_relative, f)
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_oneterm_relative.pkl", "wb") as f:
    pickle.dump(df_time_count_oneterm_relative, f)
with open("/content/drive/MyDrive/Università/Thesis/df_time_count_twoterms_relative.pkl", "wb") as f:
    pickle.dump(df_time_count_twoterms_relative, f)

---

## 7. **FIRST TWEETS**

We look for the fisrt tweet containing at least one of the keywords and hashtags we are working with.

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/date_h_openresearch.pkl", "rb") as f:
    date_h_openresearch = pickle.load(f)

In [ ]:
date_data_sharing = first_data_sharing['date'].min()
date_datasharing = first_datasharing['date'].min()
date_h_datasharing = first_h_datasharing['date'].min()
date_open_data = first_open_data['date'].min()
date_opendata = first_opendata['date'].min()
date_h_opendata = first_h_opendata['date'].min()
date_open_science = first_open_science['date'].min()
date_openscience= first_openscience['date'].min()
date_h_openscience = first_h_openscience['date'].min()
date_open_research = first_open_research['date'].min()
date_openresearch = first_openresearch['date'].min()
date_h_openresearch = first_h_openresearch['date'].min()

We print the date and time in which the first tweet was posted.

In [ ]:
print('#datasharing', date_h_datasharing, '| datasharing', date_datasharing, '| data sharing', date_data_sharing) # #datasharing 2008-05-14 10:03:45+00:00 | datasharing 2007-08-01 06:20:05+00:00 | data sharing 2007-04-11 01:57:46+00:00

In [ ]:
print('#opendata', date_h_opendata, '| opendata', date_opendata, '| open data', date_open_data) # #opendata 2008-04-09 21:14:22+00:00 | opendata 2007-03-12 14:16:20+00:00 | open data 2007-02-16 16:45:05+00:00

In [ ]:
print('#openscience', date_h_openscience, '| openscience', date_openscience, '| open science', date_open_science) # #openscience 2008-01-18 05:53:45+00:00 | openscience 2007-09-09 08:06:06+00:00 | open science 2007-07-14 17:18:12+00:00

In [ ]:
print('#openresearch', date_h_openresearch, '| openresearch', date_openresearch, '| open research', date_open_research) # #openresearch 2009-06-03 23:52:14+00:00 | openresearch 2008-12-09 18:09:05+00:00 | open research 2007-07-14 17:18:12+00:00

---

## 8. **PRE-PROCESSING DATA FILTERED BY KEYWORDS**

We repeat the previous processes applied on the entire dataset (*data_merge*) only on *filtered_keywords_df* 87256-row dataframe.

In [ ]:
filtered_keywords_df.shape[0]

87256

We first clean the df, by manually scanning tweets, in order to obtain a more precise data set.

The new dataframe contains 11959 tweets.

In [ ]:
z =  ['research', 'universit', 'academ', 'education', 'open access', 'openaccess', '#oa', ' oa ']

In [ ]:
m = ['data', 'openness', 'info', 'find', 'discover', 'stat', 'knowledg', 'datum', 'repo']

In [ ]:
research_check = filtered_keywords_df[filtered_keywords_df['text'].str.contains('|'.join(z), case= False)]
research_check.shape[0] # 29357

29357

In [ ]:
data_check = research_check[research_check['text'].str.contains('|'.join(m), case= False)]
data_check.shape[0] # 15656

15656

In [ ]:
notchecked = data_check[data_check['date'].dt.date > pd.to_datetime("2013-04-16").date()]
list_notchecked_tweet_id = notchecked.tweet_id.tolist() # 13723

In [ ]:
len(list_notchecked_tweet_id) # 13723

13723

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/list_checked_tweet_id.pkl", "rb") as f:
    list_checked_tweet_id = pickle.load(f)

In [ ]:
len(list_checked_tweet_id) # 788

788

In [ ]:
list_tweet_id = list_checked_tweet_id + list_notchecked_tweet_id
len(list_tweet_id) # 14511

14511

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/list_tweet_id.pkl", "rb") as f:
    list_tweet_id = pickle.load(f)

In [ ]:
keyword_df = filtered_keywords_df[filtered_keywords_df['tweet_id'].isin(list_tweet_id)]

In [ ]:
keyword_df.shape[0] # 11959

11959

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/keyword_df.pkl", "wb") as f:
    pickle.dump(keyword_df, f)

Thus, we repeat previous analysis made on the entire set, on this smaller subset.


First, we retrieve the number of tweets belonging to each year.

In [ ]:
df_count_per_year_filter = pd.DataFrame(keyword_df['date'].dt.year.value_counts()).sort_index().reset_index()
df_count_per_year_filter.columns = ['year', 'count']
df_count_per_year_filter

,year,count
0,2008,13
1,2009,62
2,2010,138
3,2011,172
4,2012,271
5,2013,738
6,2014,1087
7,2015,1151
8,2016,1383
9,2017,1174


In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/df_count_per_year_filter.pkl", "wb") as f:
    pickle.dump(df_count_per_year_filter, f)

Then, we split data between pre and post Covid19 era.

In [ ]:
filter_pre_covid = keyword_df[(keyword_df['date'].dt.year==2018)|(keyword_df['date'].dt.year==2019)]
filter_post_covid = keyword_df[(keyword_df['date'].dt.year==2020)|(keyword_df['date'].dt.year==2021)]

In [ ]:
filter_pre_covid.shape[0], filter_post_covid.shape[0]

(3362, 2408)

In [ ]:
with open("/content/drive/MyDrive/Università/Thesis/filter_pre_covid.pkl", "wb") as f:
    pickle.dump(filter_pre_covid, f)
with open("/content/drive/MyDrive/Università/Thesis/filter_post_covid.pkl", "wb") as f:
    pickle.dump(filter_post_covid, f)

We retrieve conversations belonging to the dataframe.

We have 11583 different conversations inside our 11966-row df, but only 649 of those have more than one tweet in our df.

In [ ]:
len(keyword_df['conversation_id'].unique())

11583

In [ ]:
conversations_df_filter = pd.DataFrame(keyword_df['conversation_id'].value_counts())

In [ ]:
conversations_list_filter = keyword_df['conversation_id'].value_counts().loc[lambda x : x>1].index.tolist()

In [ ]:
data_merge_conversations_filter = keyword_df[keyword_df['conversation_id'].isin(conversations_list_filter)]

In [ ]:
data_merge_conversations_filter.shape[0]

649

In [ ]:
# Save conversations df - full data
with open("/content/drive/MyDrive/Università/Thesis/data_merge_conversations_filter.pkl", "wb") as f:
    pickle.dump(data_merge_conversations_filter, f)